## Conclusion For SVR Models

### 1) Worst than Neural Network (R2: SVR is -1.3 vs NN is -1.0) - BOTH "arbitrarily poor models"
### 2) All show poor correlation (R-squared about -1.3), but 'polynomial' is the worst (R2 = -20); 'linear' is the best

In [1]:
# imports
import pandas as pd

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load files
file1 = "../resources/provincesSameDay.csv"
file2 = "../resources/provinces7DayAhead.csv"
file3 = "../resources/provinces14DayAhead.csv"

## Canada, Provinces, Total Cases -Same Day

In [3]:
# create df for provincesSameDay
df = pd.read_csv(file1)
df

,Unnamed: 0,date,province,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,TotalCases,DailyTotals,TotalDeaths,DailyDeaths,TotalRecovered,DailyRecovered,population
0,0,2020-02-15,ALBERTA,7.0,3.0,24.0,7.0,-2.0,-1.0,0,0,0,0,0,0,4428247
1,1,2020-02-15,BRITISH COLUMBIA,1.0,0.0,-4.0,2.0,-4.0,0.0,5,0,0,0,0,0,5120184
2,2,2020-02-15,MANITOBA,5.0,2.0,20.0,1.0,1.0,0.0,0,0,0,0,0,0,1379121
3,3,2020-02-15,NEW BRUNSWICK,-3.0,-7.0,-13.0,11.0,-3.0,1.0,0,0,0,0,0,0,780890
4,4,2020-02-15,NEWFOUNDLAND AND LABRADOR,-1.0,-2.0,-16.0,-1.0,-1.0,1.0,0,0,0,0,0,0,520437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,1401,2020-07-25,NEWFOUNDLAND AND LABRADOR,-8.0,4.0,228.0,4.0,0.0,-3.0,265,0,3,0,259,0,520437
1402,1402,2020-07-25,NOVA SCOTIA,-15.0,-10.0,327.0,-23.0,-7.0,0.0,1067,0,63,0,1003,0,978274
1403,1403,2020-07-25,ONTARIO,-22.0,-3.0,220.0,-31.0,-5.0,2.0,38543,138,2759,1,34240,140,14745040
1404,1404,2020-07-25,QUEBEC,-20.0,-6.0,255.0,-25.0,-5.0,1.0,58414,171,5666,3,50703,88,8552362


In [4]:
df.dtypes

Unnamed: 0                 int64
date                      object
province                  object
retail_and_recreation    float64
grocery_and_pharmacy     float64
parks                    float64
transit_stations         float64
workplaces               float64
residential              float64
TotalCases                 int64
DailyTotals                int64
TotalDeaths                int64
DailyDeaths                int64
TotalRecovered             int64
DailyRecovered             int64
population                 int64
dtype: object

In [5]:
# convert TotalCases & DailyTotals to float
df['TotalCases'] = df['TotalCases'].astype(float)
df['DailyTotals'] = df['DailyTotals'].astype(float)
df.dtypes

Unnamed: 0                 int64
date                      object
province                  object
retail_and_recreation    float64
grocery_and_pharmacy     float64
parks                    float64
transit_stations         float64
workplaces               float64
residential              float64
TotalCases               float64
DailyTotals              float64
TotalDeaths                int64
DailyDeaths                int64
TotalRecovered             int64
DailyRecovered             int64
population                 int64
dtype: object

## Correlate Mobility-Only Vs. Total Cases (in Alberta)

In [6]:
# Alberta only df
ab_df = df.loc[df['province'] == 'ALBERTA']
ab_df

,Unnamed: 0,date,province,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,TotalCases,DailyTotals,TotalDeaths,DailyDeaths,TotalRecovered,DailyRecovered,population
0,0,2020-02-15,ALBERTA,7.0,3.0,24.0,7.0,-2.0,-1.0,0.0,0.0,0,0,0,0,4428247
9,9,2020-02-16,ALBERTA,10.0,-2.0,28.0,3.0,-3.0,-2.0,0.0,0.0,0,0,0,0,4428247
18,18,2020-02-17,ALBERTA,-7.0,-2.0,10.0,-40.0,-67.0,15.0,0.0,0.0,0,0,0,0,4428247
27,27,2020-02-18,ALBERTA,-1.0,1.0,-2.0,-9.0,-5.0,2.0,0.0,0.0,0,0,0,0,4428247
36,36,2020-02-19,ALBERTA,3.0,4.0,21.0,-7.0,-1.0,1.0,0.0,0.0,0,0,0,0,4428247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,1367,2020-07-21,ALBERTA,-8.0,7.0,135.0,-49.0,-39.0,9.0,9728.0,141.0,172,2,8363,55,4428247
1373,1373,2020-07-22,ALBERTA,-8.0,12.0,119.0,-50.0,-37.0,10.0,9861.0,133.0,174,2,8436,73,4428247
1380,1380,2020-07-23,ALBERTA,-12.0,9.0,72.0,-51.0,-38.0,10.0,9975.0,114.0,176,2,8506,70,4428247
1389,1389,2020-07-24,ALBERTA,-17.0,6.0,90.0,-48.0,-36.0,9.0,10086.0,111.0,181,5,8567,61,4428247


In [7]:
# confirm float dtypes for df_ab
ab_df.dtypes

Unnamed: 0                 int64
date                      object
province                  object
retail_and_recreation    float64
grocery_and_pharmacy     float64
parks                    float64
transit_stations         float64
workplaces               float64
residential              float64
TotalCases               float64
DailyTotals              float64
TotalDeaths                int64
DailyDeaths                int64
TotalRecovered             int64
DailyRecovered             int64
population                 int64
dtype: object

## Prep for SVR

In [8]:
# create features (X) and target (y) sets
## 6 features

# drop columns: date, province, 
X_df = ab_df.drop(columns=['Unnamed: 0','date', 'province', 'TotalCases','DailyTotals', 'TotalDeaths',
       'DailyDeaths', 'TotalRecovered', 'DailyRecovered', 'population'], inplace=False)
display(X_df)

X = X_df.values
print(X)

y = ab_df['TotalCases'].values
y = y.reshape(-1,1)
y[:5]

,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,7.0,3.0,24.0,7.0,-2.0,-1.0
9,10.0,-2.0,28.0,3.0,-3.0,-2.0
18,-7.0,-2.0,10.0,-40.0,-67.0,15.0
27,-1.0,1.0,-2.0,-9.0,-5.0,2.0
36,3.0,4.0,21.0,-7.0,-1.0,1.0
...,...,...,...,...,...,...
1367,-8.0,7.0,135.0,-49.0,-39.0,9.0
1373,-8.0,12.0,119.0,-50.0,-37.0,10.0
1380,-12.0,9.0,72.0,-51.0,-38.0,10.0
1389,-17.0,6.0,90.0,-48.0,-36.0,9.0


[[  7.   3.  24.   7.  -2.  -1.]
 [ 10.  -2.  28.   3.  -3.  -2.]
 [ -7.  -2.  10. -40. -67.  15.]
 [ -1.   1.  -2.  -9.  -5.   2.]
 [  3.   4.  21.  -7.  -1.   1.]
 [  7.   9.  11.  -6.  -1.   0.]
 [  2.   3.  18.  -1.   2.   0.]
 [  8.   7.  19.   4.   5.  -1.]
 [  4.   3.   8.   0.   3.   0.]
 [ -1.   1.   1.   3.   4.   0.]
 [  2.   3.  13.   2.   5.   0.]
 [  5.   7.   8.   1.   5.  -1.]
 [  7.  10.  13.   1.   2.   0.]
 [  8.  11.  13.   6.   4.  -1.]
 [  5.   7.   8.   4.   5.   0.]
 [  8.  10.  13.   4.   4.  -1.]
 [  4.   7.   3.   1.   5.   0.]
 [  5.   9.  13.   0.   5.   0.]
 [  4.   7.   4.   0.   5.   0.]
 [  6.   9.   2.   0.   4.   0.]
 [  4.   9.   9.   3.   6.   0.]
 [ -2.   2. -11.  -6.   3.   2.]
 [ -1.   3. -10.  -4.   2.   1.]
 [  2.   7.  16.  -1.   4.   0.]
 [  2.   5.   9.  -4.   4.   1.]
 [  3.  11.  16.  -5.   4.   1.]
 [  4.  29.  -6.  -8.   2.   1.]
 [ -8.  30. -28. -17.  -3.   5.]
 [-19.  12. -45. -26.  -5.   7.]
 [-18.   9. -28. -25.  -7.   5.]
 [-11.  31

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [9]:
# create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=5)

In [10]:
# check number of samples (n_samples)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

153
9
153
9


In [11]:
# ensure train & test sets are correct shape
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print(X_train)
print("==============")
print(X_test)
print("==============")
print(y_train)
print("==============")
print(y_test)

[[  2.   7.  16.  -1.   4.   0.]
 [-17.   6.  90. -48. -36.   9.]
 [-57. -33. -51. -69. -43.  14.]
 [-51. -16. -49. -71. -52.  25.]
 [-14.  -1. 117. -36.  -8.   1.]
 [-15.   3.  90. -52. -39.  13.]
 [-63. -40. -43. -67. -46.  14.]
 [-20.  -7.  40. -40. -10.   4.]
 [-41. -13. -11. -65. -50.  19.]
 [-41. -11.  18. -64. -51.  20.]
 [-47. -12. -24. -68. -51.  22.]
 [-47. -17. -26. -69. -52.  21.]
 [-31.   3.  57. -58. -46.  17.]
 [-47. -11. -25. -66. -50.  22.]
 [-55. -21. -40. -63. -40.  16.]
 [-47. -13. -49. -71. -53.  23.]
 [-39.  -5.  19. -64. -51.  20.]
 [  7.  10.  13.   1.   2.   0.]
 [-20.   1.  95. -35. -10.   5.]
 [  3.  11.  16.  -5.   4.   1.]
 [-16.   3.  42. -53. -39.  13.]
 [-29.   3.  54. -43. -20.   9.]
 [-10.  12. 100. -51. -38.  12.]
 [-40. -11. -27. -66. -50.  19.]
 [-38.   4.  24. -50. -30.  10.]
 [-11.  10.  54. -51. -37.  12.]
 [-36.  -2.  20. -62. -49.  19.]
 [-11.   4.  78. -49. -37.  12.]
 [-32.  -3. -43. -66. -47.  21.]
 [  2.   3.  18.  -1.   2.   0.]
 [-45.  -8

## SVR, kernel: rbf

In [12]:
# create SVR model
n_samples, n_features = 153, 6
rng = np.random.RandomState(5)
y_train = rng.randn(n_samples)
X_train = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.1))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [13]:
# make predictions using test data
# n_samples = 9, n_features = 6
y_pred = regr.predict(X_test)

# return R-squared for the prediction
regr.score(X_test, y_test)

-1.3582034894815065

## SVR, kernel: linear

In [14]:
# create SVR model
n_samples, n_features = 153, 6
rng = np.random.RandomState(5)
y_train = rng.randn(n_samples)
X_train = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(kernel='linear', C=1.0, epsilon=0.1))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(kernel='linear'))])

In [15]:
# make predictions using test data
# n_samples = 9, n_features = 6
y_pred = regr.predict(X_test)

# return R-squared for the prediction
regr.score(X_test, y_test)

-1.354337411851951

## SVR, kernel: polynomial

In [17]:
# create SVR model
n_samples, n_features = 153, 6
rng = np.random.RandomState(5)
y_train = rng.randn(n_samples)
X_train = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(kernel='poly', degree=3, gamma='scale', coef0=0.0, C=1.0, epsilon=0.1))
regr.fit(X_train, y_train)

# make predictions using test data
# n_samples = 9, n_features = 6
y_pred = regr.predict(X_test)

# return R-squared for the prediction
regr.score(X_test, y_test)

-20.363854025157263

## SVR, kernel: sigmoid

In [20]:
# create SVR model
n_samples, n_features = 153, 6
rng = np.random.RandomState(5)
y_train = rng.randn(n_samples)
X_train = rng.randn(n_samples, n_features)
regr = make_pipeline(StandardScaler(), SVR(kernel='sigmoid', gamma='scale', coef0=0.0, C=1.0, epsilon=0.1))
regr.fit(X_train, y_train)

# make predictions using test data
# n_samples = 9, n_features = 6
y_pred = regr.predict(X_test)

# return R-squared for the prediction
regr.score(X_test, y_test)

-1.3570674287623685